# Concentration dependent elastic constants

Here we demonstrate the new `SQSElasticConstantsList` job, which gets elastic constants as a function of chemical composition.

This job works by using special quasirandom structures (implemented by Dominik Gehringer and Jan Janssen) to create cells with the requested chemical composition from a base crystal structure. These are then evaluated for their elastic constants (implemented by Yuri Lysogorskiy). These are fully relaxed and the basic output is the chemical composition and full 6x6 elastic matrix. (More detailed output is available from the underlying `SQSElasticConstants` job stored in HDF5, but user-friendly bindings are not currently implemented.)

These components were assembled together in a single job (implemented by Liam Huber) that takes a list of dictionaries giving the desired chemical compositions (e.g. `{'Al': 0.95, 'Mg': 0.04, '0': 0.01}` for 95% Aluminium 4% Magnesium and 1% vacancies) and three reference jobs: a `ref_ham` which says how the structures should be interpreted for forces and energies (e.g. a Lammps job with a particular potential) -- the structure of this job provides the base crystal upon which the chemical compositions are imposed -- and `ref_sqs` and `ref_elastic` which hold the input parameters for the SQS and elastic constant calculation components.

The current implementation only facilitates modal run modes and is not heavily tested. The example below takes less than 30 minutes on a single core, so for the purposes of demonstration this is not a serious limitation.

In [1]:
%matplotlib inline

In [2]:
from pyiron import Project
import numpy as np
import matplotlib.pyplot as plt
from elastic import ElasticMatrixJob
from damaskjob import DAMASKjob
import seaborn as sns
import yaml

In [3]:
pr = Project('elastic_pipeline')
# pr.remove_jobs_silently(recursive=True)

## Run the calculation

Our test system will be Al with dilute Mg (up to 2%). Here we define the reference jobs, give our desired compositions.

Feedback on the most user-friendly way to take chemical composition input (while still being fully flexible in allowed compositions!) is welcome -- this code is still in development.

In [4]:
lmp_mini = pr.create_job(pr.job_type.Lammps, 'lammps_mini')
lmp_mini.structure = pr.create_ase_bulk('Al', cubic=True).repeat(5)
lmp_mini.potential = '2009--Mendelev-M-I--Al-Mg--LAMMPS--ipr1'
lmp_mini.calc_minimize(pressure=0.0)
lmp_mini.run()

The job lammps_mini was saved and received the ID: 1


In [5]:
sqs_job_al = pr.create_job(
    job_type=pr.job_type.SQSJob, 
    job_name="sqs_job_al"
)
sqs_job_al.structure = lmp_mini.get_structure()
sqs_job_al.input['iterations'] = 1000
master_al = sqs_job_al.create_job(
    job_type=pr.job_type.SQSMaster, 
    job_name="master_al"
)

master_al.input["fraction_lst"] = [0.005, 0.01, 0.015, 0.02]
master_al.input["species_one"] = "Mg"
master_al.input["species_two"] = "Al"
master_al.run()

The job master_al was saved and received the ID: 2
The job sqs_0_005 was saved and received the ID: 3
Mole fractions were corrected to: {'Mg': 0.004, 'Al': 0.996}
Threads used: 1
The job sqs_0_01 was saved and received the ID: 4
Threads used: 1
The job sqs_0_015 was saved and received the ID: 5
Mole fractions were corrected to: {'Mg': 0.016, 'Al': 0.984}
Threads used: 1
The job sqs_0_02 was saved and received the ID: 6
Threads used: 1


In [6]:
concentration_lst = [0.0] + [0.005, 0.01, 0.015, 0.02]

In [7]:
structure_lst = [lmp_mini.get_structure()] + master_al.list_of_structures

In [8]:
[s.get_chemical_formula() for s in structure_lst]

['Al500', 'Al498Mg2', 'Al495Mg5', 'Al492Mg8', 'Al490Mg10']

In [9]:
for s in structure_lst:
    lmp_template = pr.create_job(pr.job_type.Lammps, 'lammps_tmp')
    lmp_template.potential = '2009--Mendelev-M-I--Al-Mg--LAMMPS--ipr1'
    lmp_template.server.run_mode.interactive = True
    lmp_template.interactive_enforce_structure_reset = True
    lmp_template.structure = s
    elastic = lmp_template.create_job(ElasticMatrixJob, 'elastic_' + s.get_chemical_formula())
    elastic.input['num_of_points'] = 5
    elastic.input['fit_order'] = 2
    elastic.input['eps_range'] = 0.005
    elastic.input['relax_atoms'] = True
    elastic.input['sqrt_eta'] = True
    elastic.run()

The job elastic_Al500 was saved and received the ID: 7
The job elastic_Al500_lammps_tmp was saved and received the ID: 8


/srv/conda/envs/notebook/lib/python3.7/site-packages/pyiron_atomistics/lammps/interactive.py:376: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  "Warning: setting upper trangular matrix might slow down the calculation"


The job elastic_Al498Mg2 was saved and received the ID: 9
The job elastic_Al498Mg2_lammps_tmp was saved and received the ID: 10
The job elastic_Al495Mg5 was saved and received the ID: 11
The job elastic_Al495Mg5_lammps_tmp was saved and received the ID: 12
The job elastic_Al492Mg8 was saved and received the ID: 13
The job elastic_Al492Mg8_lammps_tmp was saved and received the ID: 14
The job elastic_Al490Mg10 was saved and received the ID: 15
The job elastic_Al490Mg10_lammps_tmp was saved and received the ID: 16


## post-processing the data

The output of the job is stored to HDF5. To verify this, and since the actual calculation may be done at a different time than the results are analyzed, we demonstrate this by loading the job.

There is also a little bit of annoying post-processing to get it all formatted as nice numpy arrays. Some of this is reasonable (like the need to explicitly add 0% Mg to the first chemistry -- we wouldn't want to list *everything that's not there*!), some is "not my fault" (like the way the InputList class mangles our chemistry dictionary on saving it to HDF), and some needs nicer helper functions in the future (like the ability to explicity ask for "C12" and get it by that string).

In [10]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,Al500,lammps_mini,/lammps_mini,/home/jovyan/,elastic_pipeline/,2021-01-29 16:24:17.349208,2021-01-29 16:24:18.185689,0.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dil4jnxmc#1,Lammps,0.1,None,NaN
1,2,finished,Al500,master_al,/master_al,/home/jovyan/,elastic_pipeline/,2021-01-29 16:24:20.458827,2021-01-29 16:24:30.522151,10.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dil4jnxmc#1#4/4,SQSMaster,0.0.1,None,NaN
2,3,finished,Al498Mg2,sqs_0_005,/sqs_0_005,/home/jovyan/,elastic_pipeline/master_al_hdf5/,2021-01-29 16:24:21.061644,2021-01-29 16:24:22.179395,1.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dil4jnxmc#1,SQSJob,0.1,None,2.0
3,4,finished,Al495Mg5,sqs_0_01,/sqs_0_01,/home/jovyan/,elastic_pipeline/master_al_hdf5/,2021-01-29 16:24:22.798986,2021-01-29 16:24:24.066690,1.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dil4jnxmc#1,SQSJob,0.1,None,2.0
4,5,finished,Al492Mg8,sqs_0_015,/sqs_0_015,/home/jovyan/,elastic_pipeline/master_al_hdf5/,2021-01-29 16:24:24.726199,2021-01-29 16:24:28.492472,3.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dil4jnxmc#1,SQSJob,0.1,None,2.0
5,6,finished,Al490Mg10,sqs_0_02,/sqs_0_02,/home/jovyan/,elastic_pipeline/master_al_hdf5/,2021-01-29 16:24:29.334604,2021-01-29 16:24:30.497905,1.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dil4jnxmc#1,SQSJob,0.1,None,2.0
6,7,finished,Al500,elastic_Al500,/elastic_Al500,/home/jovyan/,elastic_pipeline/,2021-01-29 16:24:33.884426,2021-01-29 16:24:42.685241,8.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dil4jnxmc#1#0,ElasticMatrixJob,0.0.1,None,NaN
7,8,finished,Al500,elastic_Al500_lammps_tmp,/elastic_Al500_lammps_tmp,/home/jovyan/,elastic_pipeline/,2021-01-29 16:24:35.265324,2021-01-29 16:24:41.434139,6.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dil4jnxmc#1,Lammps,0.1,None,7.0
8,9,finished,Al498Mg2,elastic_Al498Mg2,/elastic_Al498Mg2,/home/jovyan/,elastic_pipeline/,2021-01-29 16:24:43.134409,2021-01-29 16:25:14.774797,31.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dil4jnxmc#1#0,ElasticMatrixJob,0.0.1,None,NaN
9,10,finished,Al498Mg2,elastic_Al498Mg2_lammps_tmp,/elastic_Al498Mg2_lammps_tmp,/home/jovyan/,elastic_pipeline/,2021-01-29 16:24:44.730654,2021-01-29 16:25:13.887781,29.0,pyiron@jupyter-jan-2djanssen-2dpyiron-2ddamask-2dil4jnxmc#1,Lammps,0.1,None,9.0


In [11]:
def filter_function(job): 
    return "elastic_" in job.job_name and "lammps_tmp" not in job.job_name

In [12]:
def get_c11(job):
    return job["output/elasticmatrix"]["C"][0,0]

def get_c12(job):
    return job["output/elasticmatrix"]["C"][0,1]

def get_c44(job):
    return job["output/elasticmatrix"]["C"][3,3]

In [13]:
table = pr.create_table()

In [14]:
table.filter_function = filter_function
table.add["C11"] = get_c11
table.add["C12"] = get_c12
table.add["C44"] = get_c44

In [15]:
table.run()

 20%|██        | 1/5 [00:00<00:00,  6.80it/s]

The job table was saved and received the ID: 17


100%|██████████| 5/5 [00:02<00:00,  1.81it/s]


In [16]:
df_data = table.get_dataframe()
df_data

,job_id,C11,C12,C44
0,7,110.193270,61.752630,32.746691
1,9,111.877929,62.841568,33.245410
2,11,113.162758,64.525713,34.087742
3,13,114.305713,66.200398,34.932160
4,15,115.631716,67.369169,35.499085


## Analysis

Finally, let's use numpy to make a polynomial fit of our elastic constants to alloying concentration and visualize the results.

Note: pure-Al data lines up very nicely with the pure Al elastic constants [reported on the NIST website for this potential](https://www.ctcms.nist.gov/potentials/entry/2009--Mendelev-M-I-Asta-M-Rahman-M-J-Hoyt-J-J--Al-Mg/2009--Mendelev-M-I--Al-Mg--LAMMPS--ipr1.html). I don't have a good sense for exactly what <2% Mg should be doing, but at least there are no sharp changes. Sanity check passed.

In [17]:
concentrations = np.array(concentration_lst)

In [18]:
C11_data = df_data["C11"].values
C12_data = df_data["C12"].values
C44_data = df_data["C44"].values

In [19]:
fit_degree = 2
C11_fit = np.polyfit(concentrations, C11_data, deg=fit_degree)
C12_fit = np.polyfit(concentrations, C12_data, deg=fit_degree)
C44_fit = np.polyfit(concentrations, C44_data, deg=fit_degree)
c = np.linspace(concentrations.min(), concentrations.max(), 100)  # 'Continuous' concentration

In [20]:
# p(x) = p[0] * x**deg + ... + p[deg]
print(C11_fit)
print(C12_fit)
print(C44_fit)

[-2454.81759608   315.18989324   110.25060101]
[429.16128807 283.25494384  61.64097186]
[395.70293493 135.91668804  32.68369521]


In [21]:
for color, data, fit, label in zip(
    sns.color_palette(n_colors=3), 
    [C11_data, C12_data, C44_data], 
    [C11_fit, C12_fit, C44_fit],
    ['C11', 'C12', 'C44']
):
    plt.scatter(concentrations, data, color=color)
    plt.plot(c, np.polyval(fit, c), color=color, label=label)
plt.xlabel('Mg concentration')
plt.ylabel('Elastic constant (GPa)')
plt.legend()
plt.show()

# DAMASK Simulations
The steps include, defining:
- geometries
- grains
- loading
- material properties based on the atomistically-evaluated 
- perfoming simulations

In [22]:
concentrations[::2]

array([0.  , 0.01, 0.02])

In [23]:
with open('Examples/material.yaml') as f:
    material = yaml.load(f, Loader=yaml.FullLoader)
with open('Examples/tensionX.load') as f:
    tension = f.readlines()

In [24]:
job_lst = []
for i,c in enumerate(concentrations[::2]):
    job = pr.create_job(DAMASKjob, "damask_" + 'mg_{}'.format(int(c*100)))
    job.material = material
    job.tension = tension
    job.input['C11'] = float(np.polyval(C11_fit, c))
    job.input['C12'] = float(np.polyval(C12_fit, c))
    job.input['C44'] = float(np.polyval(C44_fit, c))
    job.run()
    job_lst.append(job)

The job damask_mg_0 was saved and received the ID: 18
The job damask_mg_1 was saved and received the ID: 19
The job damask_mg_2 was saved and received the ID: 20


## Post-processing DAMASK simulations
- Von-Mises stress for each case is calculated
- The Stress-strain data is evaluated for each case


In [25]:
stresses, strains = [], []
for job in job_lst: 
    damask_results = job.output
    damask_results.add_stress_Cauchy()
    damask_results.add_stretch_tensor()
    damask_results.add_equivalent_Mises('sigma')
    damask_results.add_equivalent_Mises('epsilon_V^0.0(F)')
    damask_results.add_calculation('avg_sigma',"np.average(#sigma_vM#)")
    damask_results.add_calculation('avg_epsilon',"np.average(#epsilon_V^0.0(F)_vM#)")
    damask_results.save_VTK(['sigma','epsilon_V^0.0(F)','sigma_vM','epsilon_V^0.0(F)_vM'])
    stresses.append(job.eval_stress())
    strains.append(job.eval_strain())

 ██████████████████████████████████████████████████ 100% ETA 0:00:00
 ██████████████████████████████████████████████████ 100% ETA 0:00:00
 ██████████████████████████████████████████████████ 100% ETA 0:00:00


No matching dataset found, no data was added.


 ██████████████████████████████████████████████████ 100% ETA 0:00:00
 ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   0% ETA n/a

No matching dataset found, no data was added.


 ██████████████████████████████████████████████████ 100% ETA 0:00:00


KeyError: "Unable to open object (object 'avg_epsilon' doesn't exist)"

## Visualization
### Von-Mises Stress

In [ ]:
import os

In [ ]:
import pyvista as pv
inc=20
mesh = pv.read(os.path.basename(job.output_file.split('.')[0]) + f'_inc{inc}.vtr')  #enter the file name you wish to view

In [ ]:
from itkwidgets import view
import itk
mesh

In [ ]:
pl = pv.PlotterITK()
pl.add_mesh(mesh)
pl.show()

### Stress-strain curve

In [ ]:
%matplotlib inline

In [ ]:
i = 0;
colors = ['r','g','b']
#for color, strain, stress, c in zip(colors, strains, stresses, concentrations):
plt.plot(strains[0], stresses[0], 'r', linewidth=2, label = 'Mg={}%'.format(0*100))
plt.plot(strains[1], stresses[1], 'b', linestyle="None",marker = 'o' ,label = 'Mg={}%'.format(0.01*100))
plt.plot(strains[1], stresses[2], 'g', linestyle="None",marker = '*' ,label = 'Mg={}%'.format(0.02*100))
#plt.ylim(90, 95)
plt.grid(True)
plt.xlabel('$Strain$')
plt.ylabel('Stress- MPa')
plt.legend( loc='lower right')
plt.title("Stress-Strain")
 #   i= i+1;